In [1]:
import numpy as np
import random
import tensorflow as tf
import datetime
import pandas as pd

In [2]:
print(tf.test.gpu_device_name())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
df_songs = pd.read_csv("songs_details.csv")
df_english = df_songs[df_songs["lang_lyrics"] == "en"]
print("Lyrics have on average:",int(df_english["lyrics_cleaned"].apply(lambda x: len(x)).mean()), "characters")

texts = ""
for song in df_english.index:
    lyrics = df_english.loc[song,"lyrics_cleaned"]
    texts = str(lyrics) + texts
    
chars = sorted(list(set(texts)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Lyrics have on average: 2252 characters
total chars: 55


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(texts) - maxlen, step):
    sentences.append(texts[i: i + maxlen])
    next_chars.append(texts[i + maxlen])
print('Number of sequences:', len(sentences), "\n")

print(sentences[:10], "\n")
print(next_chars[:10])

Number of sequences: 223749 

[' uh fuck i dont fuck with you you little', ' fuck i dont fuck with you you little st', 'ck i dont fuck with you you little stupi', 'i dont fuck with you you little stupid a', 'ont fuck with you you little stupid ass ', ' fuck with you you little stupid ass bit', 'ck with you you little stupid ass bitch ', 'with you you little stupid ass bitch i a', 'h you you little stupid ass bitch i aint', 'ou you little stupid ass bitch i aint fu'] 

[' ', 'u', 'd', 's', 'b', 'c', 'i', 'i', ' ', 'c']


In [5]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint
import random
import sys
import io

Using TensorFlow backend.


In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked for specified epochs. Prints generated text.
    # Using epoch+1 to be consistent with the training epochs printed by Keras
    if epoch+1 == 1 or epoch+1 == 15:
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(texts) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = texts[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()
    else:
        print()
        print('----- Not generating text after Epoch: %d' % epoch)

generate_text = LambdaCallback(on_epoch_end=on_epoch_end)

In [9]:
# define the checkpoint
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
model.fit(x, y,
              batch_size=128,
              epochs=15,
              verbose=2,
              callbacks=[generate_text, checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 - 94s - loss: 1.8473

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "t you try again boy you gotta do it righ"
t you try again boy you gotta do it right in the might and i cant they want the might the mind me i cant the might me i know i cant the mind i talkin and i still the mind i know me i not a mind i cant the might i cant thing i think im think i talk the mind i aint man the mind i think im think im the mind this the stare the mind i know i still the mind i cant thinking i aint that i cant gotta the mind mind me i cant they thinking in the 
----- diversity: 0.5
----- Generating with seed: "t you try again boy you gotta do it righ"
t you try again boy you gotta do it right go to me san me nothin give up pay out these going im all nate i how the man the mind that and me i cant this she in the might a give up and im truight girl and prop i that misting they will in to im like come im s

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


qo be wju   ke s t zt th t  me thre the sout  nin thek i  i the me droceui git i goe me the jume ige cn i  anij sz   ca t  cl e t h foren chtim the  ther c me kult ou bu fe th the i i snve the s juju thegh  th  de jung i th i me he t th  wahe th u w t qu te thuc  therar  he xe fot h m th nhe a khe t i go   bet  we thad aj a y i ke tum nea conl y ux tjut tht up   ck 
----- diversity: 0.5
----- Generating with seed: " but i feel it still ooh woo ooh woo but"
 but i feel it still ooh woo ooh woo but  he ded inl ro lhd d me nut s l the h re wh nv  ea th0ee  i  ingr i p  utj th nort up an reexec st jun mi ce me ye silgot in p in the  h  wh do in  gou i  squd t the ande tung he toki y they eem wi ssast m te ferd me ir conge t lei s g ano in the jut i  the   withe you jut i p me  th m  hex  mes my al caithe minde helufue m n a atheat ade y a me theyejx nea i swe  as tar g men romr oj s iuc s tx 
----- diversity: 1.0
----- Generating with seed: " but i feel it still ooh woo ooh woo but"
 but i